In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
import keras
import scipy as sp
import scipy.signal
import tensorflow as tf
import tensorflow.keras.losses as losses
import tensorflow.keras.optimizers as optimizers
import scipy.io as sio
import numpy as np
import sklearn.metrics as metrics 
import matplotlib.pyplot as plt
import statistics

from tensorflow_addons.metrics import RSquare
from tensorflow.keras.models import Sequential
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from wandb.keras import WandbCallback
import wandb
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
F1_test=8
D_test=2
F2_test=16
Drop_test=0 #originally 0.5
KernLength_test=64 #Originally 64 
batch_test=16

N_chan=31
N_samples_long=250
# N_samples_short=251
print(tf.config.list_physical_devices('GPU'))
MIN_LOSS_FOUND=100
MAX_R2_FOUND=-100
MAX_CORR_COEF_FOUND=-100

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs=None):
        old_weights=[]
        for idx in range(8):
            old_weights.append(model.trainable_variables[0].numpy()[0,:,0,idx])
      
        old_weights=np.stack(old_weights)
        lista.append(old_weights)

In [ ]:
#SINGLE ROI
SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
for temp_sub in SUBJECTS:
    lista=[]
    sub="{:02d}".format(temp_sub)

    #Define loss function
    loss_fn= tf.keras.losses.MeanSquaredError()
    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    MSE_filters_mean=[]
    NUM_ROI=1

    print("SUBJECT: "+ str(sub))
    
    X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
    y= sio.loadmat("FINAL_regression/right_SMA/input/sub-"+str(sub)+"_compiled_bold_non_norm_z_score_run_clipped.mat")["compiled_bold_non_norm_z_score_run_clipped"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir0= os.path.join("FINAL_regression/right_SMA/output", "sub_"+str(sub))
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################
    Fs=125
    EPOCHS=800
    lr_test=1e-4
    batch_test=64
    decay_test=0
    kern_test=0.15

    n_folds = 5
    seed = 21
    shuffle_test = True
    r2=[]
    adj_r2=[]
    corr_coef=[]
    final_val_loss=[]
    stats=[]

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(NUM_ROI, name = 'dense',
                             kernel_constraint = max_norm(kern_test)))
#     model.add(Activation('softmax', name = 'softmax'))


          # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= lr_test,
        weight_decay= decay_test
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=[RSquare()])

        evaluation.append(model.fit(X_train, y_train, batch_size=batch_test,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1, 
                  callbacks=[CustomCallback()]))

        iteration.append(model)
        #SALVO R2 e Adjusted R2 score per fold
        modello= model.predict(X_test)

        temp_r2=[]
        temp_adj_r2=[]
        temp_r2=r2_score(y_test, modello)

        n= y_test.shape[0]
        p= N_chan*N_samples_long
        temp_adj_r2= 1-((1-temp_r2)*((n-1)/(n-p)))

        temp_corr_coef=[]
        modello=modello.flatten()
        temp_corr_coef=np.corrcoef(y_test, modello)[0,1]

        r2.append(temp_r2)
        adj_r2.append(temp_adj_r2)
        corr_coef.append(temp_corr_coef)

        # PLOTTO EFFETTIVA REGRESSIONE VISIVAMENTE
        plt.figure()
        plt.figure(figsize=(25,5))
        plt.plot(y_test)
        plt.plot(model.predict(X_test))
    #     plt.plot(y_test-model.predict(X_test))
        plt.savefig(dir1+"/comparison_kfold_"+str(count)+"_zscored_run_clipped")

        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": temp})
            plt.close()

            neil_x, neil_y=sp.signal.welch(model.trainable_variables[0].numpy()[0,:,0,lallo], Fs, nperseg=64)
            plt.figure()
            plt.title("Spectrogram_"+str(lallo))
            plt.plot(neil_x, neil_y)
            plt.savefig(dir2+"/ALL_spectrogram_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
            plt.close()

        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/ALL_spat_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": nump})

    # print("--------------------------------------- AVERAGE MEAN AND VARIANCE ACROSS FOLDS ---------------------------------------")
    # print(np.mean(val_mean_squared_error))
    # print(np.var(val_mean_squared_error))

    # PLOTTO LOSS E VALIDATION LOSS, MEAN SQUARED ERROR E VALIDATION MEAN SQUARED ERROR

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['r_square'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['r_square'])
        if (np.min(evaluation[idx].history['val_r_square'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_r_square'])
        if (np.max(evaluation[idx].history['r_square'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['r_square'])
        if (np.max(evaluation[idx].history['val_r_square'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_r_square'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for kfold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()


    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    # inizio=0;
    # for idx in range(n_folds):
    #     mse_filter_min= min(MSE_filters_mean)
    #     mse_filter_max= max(MSE_filters_mean)
    #     fine=(idx+1)*EPOCHS

    #     plt.figure()
    #     plt.plot(MSE_filters_mean[inizio:fine],'b-');

    #     plt.xlabel('Epoch')
    #     plt.ylabel('mean mse filters')
    #     plt.title('mean_mse_temporal_filters across epochs for kfold: '+str(idx))
    #     plt.ylim([mse_filter_min, mse_filter_max])
    # #     plt.legend()

    #     plt.savefig(dir1+"/mean_mse_filters_kfold_"+str(idx))
    #     plt.close()
    #     inizio=fine

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['r_square']
        loss_vec_test= evaluation[idx].history['val_r_square']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('r_square')
        plt.title('r_square across epochs for kfold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/r_square_kfold_"+str(idx))
        plt.close()

    for idx in range(n_folds):
        temp_val_loss=[]
        temp_val_loss= evaluation[idx].history['val_loss'][-1] #with this indexing i select the last element of the list
        final_val_loss.append(temp_val_loss)

    gianfranco= model.predict(X)

    plt.figure()
    plt.figure(figsize=(25,5))
    plt.plot(y, 'b', label='true')
    plt.plot(gianfranco, 'r', label='predicted')
    plt.legend()
    plt.savefig(dir1+"/total_comparison_zscored_run_clipped")
    plt.savefig(dir0+"/total_comparison_zscored_run_clipped")

    sio.savemat(dir1+"/regression_results_zscored_run_clipped.mat", {"array": gianfranco})

    sio.savemat(dir1+"/r2_score_zscored_run_clipped.mat", {"array": r2})
    sio.savemat(dir1+"/adjusted_r2_score_zscored_run_clipped.mat", {"array": adj_r2})
    sio.savemat(dir1+"/corr_coef_zscored_run_clipped.mat", {"array": corr_coef})
    sio.savemat(dir1+"/final_val_loss_zscored_run_clipped.mat", {"array": final_val_loss})

    avg_final_loss= np.mean(final_val_loss)
    var_final_loss=statistics.pstdev(final_val_loss)
    avg_r2= np.mean(r2)
    var_r2=statistics.pstdev(r2)
    avg_corr_coef= np.mean(corr_coef)
    var_corr_coef= statistics.pstdev(corr_coef)
    
    stats.append(avg_final_loss)
    stats.append(var_final_loss)
    stats.append(avg_r2)
    stats.append(var_r2)
    stats.append(avg_corr_coef)
    stats.append(var_corr_coef)
    sio.savemat(dir0+"/stats_zscored_run_clipped.mat", {"array": stats})
    
####################################################################################################################################

    mse_epoca=[]
    for epoca in range(EPOCHS*n_folds):
        mse_filtro=[]
        for filtro in range(8):
            curr= lista[epoca][filtro][:]
            precedente= lista[epoca-1][filtro][:]    
            mse_filtro.append(((curr-precedente)**2).mean(axis=None))

        mse_epoca.append(np.mean(mse_filtro))    
        
    inizio=0
    for idx in range(n_folds):
        mse_filter_min= min(mse_epoca)
        mse_filter_max= max(mse_epoca)
        fine=(idx+1)*EPOCHS

        plt.figure()
        plt.plot(mse_epoca[inizio+1:fine],'b-');

        plt.xlabel('Epoch')
        plt.ylabel('mean mse filters')
        plt.title('mean_mse_temporal_filters across epochs for kfold: '+str(idx))
    #     plt.ylim([mse_filter_min, mse_filter_max])
        plt.legend()
        sio.savemat(dir1+"/mean_mse_filter_kfold_"+str(idx)+".mat", {"array": mse_epoca[inizio+1:fine]})

        plt.savefig(dir1+"/mean_mse_filters_kfold_"+str(idx))
        plt.close()
        inizio=fine

# ALL

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs=None):
        old_weights=[]
        for idx in range(8):
            old_weights.append(model.trainable_variables[0].numpy()[0,:,0,idx])
      
        old_weights=np.stack(old_weights)
        lista.append(old_weights)

In [ ]:
#ALL ROI
lista=[]

#Define loss function
loss_fn= tf.keras.losses.MeanSquaredError()
evaluation=[]
iteration=[]
confusion_matrix_x_test=[]
confusion_matrix_y_test= []
validation_acc=[]
MSE_filters_mean=[]
NUM_ROI=1

X=np.load("input/RAW_regression_all_subs_float32.npy")
y= sio.loadmat("FINAL_regression/right_SMA/input/compiled_bold_non_norm_all_z_score_run_clipped.mat")["compiled_bold_non_norm_all_z_score_run_clipped"]

X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y=y.flatten()
print("Shape for X: " + str(X.shape))
print("Shape for y: "+str(y.shape))

X= X[762:,:,:,:]
y= y[762:]
print("New shape for X: " + str(X.shape))
print("New shape for y: "+str(y.shape))
    
dir0= os.path.join("FINAL_regression/right_SMA/output", "all")
os.mkdir(dir0)
dir1=os.path.join(dir0, "comparison")
dir2=os.path.join(dir0, "temporal_convolution")
dir3=os.path.join(dir0, "spatial_convolution")
os.mkdir(dir1)
os.mkdir(dir2)
os.mkdir(dir3)

################################################################################################################################################################################################################################################################################
Fs=125
EPOCHS=800
lr_test=1e-4
batch_test=64
decay_test=0
kern_test=0.15

n_folds = 5
seed = 21
shuffle_test = True
r2=[]
adj_r2=[]
corr_coef=[]
final_val_loss=[]
stats=[]

kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
count=0
sommatoria=0

for train_index, test_index in kfold.split(X):
    count=count+1

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print("Train index for this split: "+ str(train_index)) 
    print("Number of samples for train set: "+str(train_index.shape[0]))
    print("Test index for this split: "+ str(test_index))
    print("Number of samples for test set: "+str(test_index.shape[0]))

    # Define the model architecture - 

    model=Sequential()

    ##################################################################

    model.add(Conv2D(8, (1, 64), padding = 'same',
                                   input_shape = (N_chan, N_samples_long, 1),
                                   use_bias = False, name="temporal_conv"))
    model.add(BatchNormalization(name="batchnorm_1"))
    model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                   depth_multiplier = 2,
                                   depthwise_constraint = max_norm(1.), name="spatial_conv"))
    model.add(BatchNormalization(name="batchnorm_2"))
    model.add(Activation('elu', name="activation_1"))
    model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
    model.add(Dropout(0.5, name="dropout_1"))

    model.add(SeparableConv2D(16, (1, 16),
                                   use_bias = False, padding = 'same', name="separable_conv"))
    model.add(BatchNormalization(name="batchnorm_3"))
    model.add(Activation('elu', name="activation_2"))
    model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
    model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

    model.add(Flatten(name = 'flatten'))

    model.add(Dense(NUM_ROI, name = 'dense',
                         kernel_constraint = max_norm(kern_test)))
#     model.add(Activation('softmax', name = 'softmax'))


      # Define the optimizer
    optimizer= optimizers.Adam(
    learning_rate= lr_test,
    weight_decay= decay_test
    )
    model.compile(optimizer=optimizer,
                   loss=loss_fn,
                   metrics=[RSquare()])

    evaluation.append(model.fit(X_train, y_train, batch_size=batch_test,
              epochs=EPOCHS, 
              validation_data=(X_test, y_test), 
              verbose=2, workers=1, 
              callbacks=[CustomCallback()]))

    iteration.append(model)
    #SALVO R2 e Adjusted R2 score per fold
    modello= model.predict(X_test)

    temp_r2=[]
    temp_adj_r2=[]
    temp_r2=r2_score(y_test, modello)

    n= y_test.shape[0]
    p= N_chan*N_samples_long
    temp_adj_r2= 1-((1-temp_r2)*((n-1)/(n-p)))

    temp_corr_coef=[]
    modello=modello.flatten()
    temp_corr_coef=np.corrcoef(y_test, modello)[0,1]

    r2.append(temp_r2)
    adj_r2.append(temp_adj_r2)
    corr_coef.append(temp_corr_coef)

    # PLOTTO EFFETTIVA REGRESSIONE VISIVAMENTE
    plt.figure()
    plt.figure(figsize=(25,5))
    plt.plot(y_test)
    plt.plot(model.predict(X_test))
#     plt.plot(y_test-model.predict(X_test))
    plt.savefig(dir1+"/comparison_kfold_"+str(count)+"_zscored_run_clipped")

    var= (model.get_layer("temporal_conv").weights)
    for lallo in range(8):
        plt.figure()
        plt.title("temp_conv_"+str(lallo))
        plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
        plt.savefig(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
        temp= (var[0][0,:,0][:,lallo]).numpy()
        sio.savemat(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": temp})
        plt.close()

        neil_x, neil_y=sp.signal.welch(model.trainable_variables[0].numpy()[0,:,0,lallo], Fs, nperseg=64)
        plt.figure()
        plt.title("Spectrogram_"+str(lallo))
        plt.plot(neil_x, neil_y)
        plt.savefig(dir2+"/ALL_spectrogram_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
        plt.close()

    var_2= (model.get_layer("spatial_conv").weights)
    reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
    for lallo in range(16):
        nump= reshaped_var_2[:,lallo].numpy()
        sio.savemat(dir3+"/ALL_spat_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": nump})

# print("--------------------------------------- AVERAGE MEAN AND VARIANCE ACROSS FOLDS ---------------------------------------")
# print(np.mean(val_mean_squared_error))
# print(np.var(val_mean_squared_error))

# PLOTTO LOSS E VALIDATION LOSS, MEAN SQUARED ERROR E VALIDATION MEAN SQUARED ERROR

#plot accuracy and loss function across epochs
epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
min_temp_loss=10
min_temp_acc=10
max_temp_loss=0
max_temp_acc=0

for idx in range(n_folds):
    if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
        min_temp_loss=np.min(evaluation[idx].history['loss'])
    if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
        min_temp_loss=np.min(evaluation[idx].history['val_loss'])
    if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
        max_temp_loss=np.max(evaluation[idx].history['loss'])
    if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
        max_temp_loss=np.max(evaluation[idx].history['val_loss'])

for idx in range(n_folds):
    if (np.min(evaluation[idx].history['r_square'])<min_temp_acc):
        min_temp_acc=np.min(evaluation[idx].history['r_square'])
    if (np.min(evaluation[idx].history['val_r_square'])<min_temp_acc):
        min_temp_acc=np.min(evaluation[idx].history['val_r_square'])
    if (np.max(evaluation[idx].history['r_square'])>max_temp_acc):
        max_temp_acc=np.max(evaluation[idx].history['r_square'])
    if (np.max(evaluation[idx].history['val_r_square'])>max_temp_acc):
        max_temp_acc=np.max(evaluation[idx].history['val_r_square'])

for idx in range(n_folds):
    loss_vec_train= evaluation[idx].history['loss']
    loss_vec_test= evaluation[idx].history['val_loss']

    plt.figure()
    plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
    plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss across epochs for kfold: '+str(idx))
    plt.ylim([min_temp_loss, max_temp_loss])
    plt.legend()

    plt.savefig(dir1+"/loss_kfold_"+str(idx))
    plt.close()


#plot accuracy and loss function across epochs
epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

# inizio=0;
# for idx in range(n_folds):
#     mse_filter_min= min(MSE_filters_mean)
#     mse_filter_max= max(MSE_filters_mean)
#     fine=(idx+1)*EPOCHS

#     plt.figure()
#     plt.plot(MSE_filters_mean[inizio:fine],'b-');

#     plt.xlabel('Epoch')
#     plt.ylabel('mean mse filters')
#     plt.title('mean_mse_temporal_filters across epochs for kfold: '+str(idx))
#     plt.ylim([mse_filter_min, mse_filter_max])
# #     plt.legend()

#     plt.savefig(dir1+"/mean_mse_filters_kfold_"+str(idx))
#     plt.close()
#     inizio=fine

for idx in range(n_folds):
    loss_vec_train= evaluation[idx].history['r_square']
    loss_vec_test= evaluation[idx].history['val_r_square']

    plt.figure()
    plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
    plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

    plt.xlabel('Epoch')
    plt.ylabel('r_square')
    plt.title('r_square across epochs for kfold: '+str(idx))
    plt.ylim([min_temp_acc, max_temp_acc])
    plt.legend()

    plt.savefig(dir1+"/r_square_kfold_"+str(idx))
    plt.close()

for idx in range(n_folds):
    temp_val_loss=[]
    temp_val_loss= evaluation[idx].history['val_loss'][-1] #with this indexing i select the last element of the list
    final_val_loss.append(temp_val_loss)

gianfranco= model.predict(X)

plt.figure()
plt.figure(figsize=(25,5))
plt.plot(y, 'b', label='true')
plt.plot(gianfranco, 'r', label='predicted')
plt.legend()
plt.savefig(dir1+"/total_comparison_zscored_run_clipped")
plt.savefig(dir0+"/total_comparison_zscored_run_clipped")

sio.savemat(dir1+"/regression_results_zscored_run_clipped.mat", {"array": gianfranco})

sio.savemat(dir1+"/r2_score_zscored_run_clipped.mat", {"array": r2})
sio.savemat(dir1+"/adjusted_r2_score_zscored_run_clipped.mat", {"array": adj_r2})
sio.savemat(dir1+"/corr_coef_zscored_run_clipped.mat", {"array": corr_coef})
sio.savemat(dir1+"/final_val_loss_zscored_run_clipped.mat", {"array": final_val_loss})

avg_final_loss= np.mean(final_val_loss)
var_final_loss=statistics.pstdev(final_val_loss)
avg_r2= np.mean(r2)
var_r2=statistics.pstdev(r2)
avg_corr_coef= np.mean(corr_coef)
var_corr_coef= statistics.pstdev(corr_coef)

stats.append(avg_final_loss)
stats.append(var_final_loss)
stats.append(avg_r2)
stats.append(var_r2)
stats.append(avg_corr_coef)
stats.append(var_corr_coef)
sio.savemat(dir0+"/stats_zscored_run_clipped.mat", {"array": stats})

####################################################################################################################################

mse_epoca=[]
for epoca in range(EPOCHS*n_folds):
    mse_filtro=[]
    for filtro in range(8):
        curr= lista[epoca][filtro][:]
        precedente= lista[epoca-1][filtro][:]    
        mse_filtro.append(((curr-precedente)**2).mean(axis=None))

    mse_epoca.append(np.mean(mse_filtro))    

inizio=0
for idx in range(n_folds):
    mse_filter_min= min(mse_epoca)
    mse_filter_max= max(mse_epoca)
    fine=(idx+1)*EPOCHS

    plt.figure()
    plt.plot(mse_epoca[inizio+1:fine],'b-');

    plt.xlabel('Epoch')
    plt.ylabel('mean mse filters')
    plt.title('mean_mse_temporal_filters across epochs for kfold: '+str(idx))
#     plt.ylim([mse_filter_min, mse_filter_max])
    plt.legend()
    sio.savemat(dir1+"/mean_mse_filter_kfold_"+str(idx)+".mat", {"array": mse_epoca[inizio+1:fine]})

    plt.savefig(dir1+"/mean_mse_filters_kfold_"+str(idx))
    plt.close()
    inizio=fine

# forced filtering

In [ ]:
Fs=125
b,a=sp.signal.butter(10, Wn=30, btype="lowpass", fs=Fs)

In [ ]:
class CustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
            new_weights=[]
            MSE_filters=[]
            for idx in range(8):
                old_weights= (model.trainable_variables[0].numpy()[0,:,0,idx])
                curr_new_weights=sp.signal.lfilter(b,a,old_weights)

                new_weights.append(curr_new_weights)
                MSE_filters.append( ((curr_new_weights-old_weights)**2).mean(axis=None) )


            new_weights=np.stack(new_weights)
            new_weights=np.transpose(new_weights)
            new_weights=new_weights.reshape(1,64,1,8)
            self.model.trainable_variables[0].assign(tf.Variable(new_weights, dtype=tf.float32))

            MSE_filters_mean.append(np.mean(MSE_filters))

In [ ]:
#SINGLE ROI
SUBJECTS=[1,2,3,4,5,6,7,8,9,11,12,13,14,16,17]
for temp_sub in SUBJECTS:
    lista=[]
    sub="{:02d}".format(temp_sub)

    #Define loss function
    loss_fn= tf.keras.losses.MeanSquaredError()
    evaluation=[]
    iteration=[]
    confusion_matrix_x_test=[]
    confusion_matrix_y_test= []
    validation_acc=[]
    MSE_filters_mean=[]
    NUM_ROI=1

    print("SUBJECT: "+ str(sub))
    
    X=np.load("input/sub_"+str(sub)+ "_compiled_RAW_downsampled_float32.npy")
    y= sio.loadmat("FINAL_regression/right_SMA/input/sub-"+str(sub)+"_compiled_bold_non_norm_z_score_run_clipped.mat")["compiled_bold_non_norm_z_score_run_clipped"]

    X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    y=y.flatten()
    print("New shape for X: " + str(X.shape))
    print("New shape for y: "+str(y.shape))

    dir0= os.path.join("FINAL_regression/right_SMA/output", "forced_filtering_sub_"+str(sub))
    os.mkdir(dir0)
    dir1=os.path.join(dir0, "comparison")
    dir2=os.path.join(dir0, "temporal_convolution")
    dir3=os.path.join(dir0, "spatial_convolution")
    os.mkdir(dir1)
    os.mkdir(dir2)
    os.mkdir(dir3)

    ################################################################################################################################################################################################################################################################################
    Fs=125
    EPOCHS=800
    lr_test=1e-4
    batch_test=64
    decay_test=0
    kern_test=0.15

    n_folds = 5
    seed = 21
    shuffle_test = True
    r2=[]
    adj_r2=[]
    corr_coef=[]
    final_val_loss=[]
    stats=[]

    kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
    count=0
    sommatoria=0

    for train_index, test_index in kfold.split(X):
        count=count+1

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        print("Train index for this split: "+ str(train_index)) 
        print("Number of samples for train set: "+str(train_index.shape[0]))
        print("Test index for this split: "+ str(test_index))
        print("Number of samples for test set: "+str(test_index.shape[0]))

        # Define the model architecture - 

        model=Sequential()

        ##################################################################

        model.add(Conv2D(8, (1, 64), padding = 'same',
                                       input_shape = (N_chan, N_samples_long, 1),
                                       use_bias = False, name="temporal_conv"))
        model.add(BatchNormalization(name="batchnorm_1"))
        model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                       depth_multiplier = 2,
                                       depthwise_constraint = max_norm(1.), name="spatial_conv"))
        model.add(BatchNormalization(name="batchnorm_2"))
        model.add(Activation('elu', name="activation_1"))
        model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
        model.add(Dropout(0.5, name="dropout_1"))

        model.add(SeparableConv2D(16, (1, 16),
                                       use_bias = False, padding = 'same', name="separable_conv"))
        model.add(BatchNormalization(name="batchnorm_3"))
        model.add(Activation('elu', name="activation_2"))
        model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
        model.add(Dropout(0.5, name="drpout_2"))

        model.add(Flatten(name = 'flatten'))

        model.add(Dense(1, name = 'dense',
                                 kernel_constraint = max_norm(kern_test)))
    #     model.add(Activation('softmax', name = 'softmax'))


        # Define the optimizer
        optimizer= optimizers.Adam(
        learning_rate= lr_test,
        weight_decay= decay_test
        )
        model.compile(optimizer=optimizer,
                       loss=loss_fn,
                       metrics=[RSquare()])

        evaluation.append(model.fit(X_train, y_train, batch_size=batch_test,
                  epochs=EPOCHS, 
                  validation_data=(X_test, y_test), 
                  verbose=2, workers=1, 
                  callbacks=[CustomCallback()]))

        iteration.append(model)
        #SALVO R2 e Adjusted R2 score per fold
        modello= model.predict(X_test)

        temp_r2=[]
        temp_adj_r2=[]
        temp_r2=r2_score(y_test, modello)

        n= y_test.shape[0]
        p= N_chan*N_samples_long
        temp_adj_r2= 1-((1-temp_r2)*((n-1)/(n-p)))

        temp_corr_coef=[]
        modello=modello.flatten()
        temp_corr_coef=np.corrcoef(y_test, modello)[0,1]

        r2.append(temp_r2)
        adj_r2.append(temp_adj_r2)
        corr_coef.append(temp_corr_coef)

        # PLOTTO EFFETTIVA REGRESSIONE VISIVAMENTE
        plt.figure()
        plt.figure(figsize=(25,5))
        plt.plot(y_test)
        plt.plot(model.predict(X_test))
    #     plt.plot(y_test-model.predict(X_test))
        plt.savefig(dir1+"/comparison_kfold_"+str(count)+"_zscored_run_clipped")

        var= (model.get_layer("temporal_conv").weights)
        for lallo in range(8):
            plt.figure()
            plt.title("temp_conv_"+str(lallo))
            plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
            plt.savefig(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
            temp= (var[0][0,:,0][:,lallo]).numpy()
            sio.savemat(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": temp})
            plt.close()

            neil_x, neil_y=sp.signal.welch(model.trainable_variables[0].numpy()[0,:,0,lallo], Fs, nperseg=64)
            plt.figure()
            plt.title("Spectrogram_"+str(lallo))
            plt.plot(neil_x, neil_y)
            plt.savefig(dir2+"/ALL_spectrogram_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
            plt.close()

        var_2= (model.get_layer("spatial_conv").weights)
        reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
        for lallo in range(16):
            nump= reshaped_var_2[:,lallo].numpy()
            sio.savemat(dir3+"/ALL_spat_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": nump})

    # print("--------------------------------------- AVERAGE MEAN AND VARIANCE ACROSS FOLDS ---------------------------------------")
    # print(np.mean(val_mean_squared_error))
    # print(np.var(val_mean_squared_error))

    # PLOTTO LOSS E VALIDATION LOSS, MEAN SQUARED ERROR E VALIDATION MEAN SQUARED ERROR

    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
    min_temp_loss=10
    min_temp_acc=10
    max_temp_loss=0
    max_temp_acc=0

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['loss'])
        if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
            min_temp_loss=np.min(evaluation[idx].history['val_loss'])
        if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['loss'])
        if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
            max_temp_loss=np.max(evaluation[idx].history['val_loss'])

    for idx in range(n_folds):
        if (np.min(evaluation[idx].history['r_square'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['r_square'])
        if (np.min(evaluation[idx].history['val_r_square'])<min_temp_acc):
            min_temp_acc=np.min(evaluation[idx].history['val_r_square'])
        if (np.max(evaluation[idx].history['r_square'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['r_square'])
        if (np.max(evaluation[idx].history['val_r_square'])>max_temp_acc):
            max_temp_acc=np.max(evaluation[idx].history['val_r_square'])

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['loss']
        loss_vec_test= evaluation[idx].history['val_loss']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Loss across epochs for kfold: '+str(idx))
        plt.ylim([min_temp_loss, max_temp_loss])
        plt.legend()

        plt.savefig(dir1+"/loss_kfold_"+str(idx))
        plt.close()


    #plot accuracy and loss function across epochs
    epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

    inizio=0;
    for idx in range(n_folds):
        mse_filter_min= min(MSE_filters_mean)
        mse_filter_max= max(MSE_filters_mean)
        fine=(idx+1)*EPOCHS

        plt.figure()
        plt.plot(MSE_filters_mean[inizio:fine],'b-');

        plt.xlabel('Epoch')
        plt.ylabel('mean mse filters')
        plt.title('mean_mse_temporal_filters across epochs for kfold: '+str(idx))
        plt.ylim([mse_filter_min, mse_filter_max])
#         plt.legend()
        sio.savemat(dir1+"/mean_mse_filter_kfold_"+str(idx)+".mat", {"array": MSE_filters_mean[inizio:fine]})

        plt.savefig(dir1+"/mean_mse_filters_kfold_"+str(idx))
        plt.close()
        inizio=fine

    for idx in range(n_folds):
        loss_vec_train= evaluation[idx].history['r_square']
        loss_vec_test= evaluation[idx].history['val_r_square']

        plt.figure()
        plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
        plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

        plt.xlabel('Epoch')
        plt.ylabel('r_square')
        plt.title('r_square across epochs for kfold: '+str(idx))
        plt.ylim([min_temp_acc, max_temp_acc])
        plt.legend()

        plt.savefig(dir1+"/r_square_kfold_"+str(idx))
        plt.close()

    for idx in range(n_folds):
        temp_val_loss=[]
        temp_val_loss= evaluation[idx].history['val_loss'][-1] #with this indexing i select the last element of the list
        final_val_loss.append(temp_val_loss)

    gianfranco= model.predict(X)

    plt.figure()
    plt.figure(figsize=(25,5))
    plt.plot(y, 'b', label='true')
    plt.plot(gianfranco, 'r', label='predicted')
    plt.legend()
    plt.savefig(dir1+"/total_comparison_zscored_run_clipped")
    plt.savefig(dir0+"/total_comparison_zscored_run_clipped")

    sio.savemat(dir1+"/regression_results_zscored_run_clipped.mat", {"array": gianfranco})

    sio.savemat(dir1+"/r2_score_zscored_run_clipped.mat", {"array": r2})
    sio.savemat(dir1+"/adjusted_r2_score_zscored_run_clipped.mat", {"array": adj_r2})
    sio.savemat(dir1+"/corr_coef_zscored_run_clipped.mat", {"array": corr_coef})
    sio.savemat(dir1+"/final_val_loss_zscored_run_clipped.mat", {"array": final_val_loss})

    avg_final_loss= np.mean(final_val_loss)
    var_final_loss=statistics.pstdev(final_val_loss)
    avg_r2= np.mean(r2)
    var_r2=statistics.pstdev(r2)
    avg_corr_coef= np.mean(corr_coef)
    var_corr_coef= statistics.pstdev(corr_coef)
    
    stats.append(avg_final_loss)
    stats.append(var_final_loss)
    stats.append(avg_r2)
    stats.append(var_r2)
    stats.append(avg_corr_coef)
    stats.append(var_corr_coef)
    sio.savemat(dir0+"/stats_zscored_run_clipped.mat", {"array": stats})

# all

In [ ]:
lista=[]

#Define loss function
loss_fn= tf.keras.losses.MeanSquaredError()
evaluation=[]
iteration=[]
confusion_matrix_x_test=[]
confusion_matrix_y_test= []
validation_acc=[]
MSE_filters_mean=[]
NUM_ROI=1

X=np.load("input/RAW_regression_all_subs_float32.npy")
y= sio.loadmat("FINAL_regression/right_SMA/input/compiled_bold_non_norm_all_z_score_run_clipped.mat")["compiled_bold_non_norm_all_z_score_run_clipped"]

X= X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
y=y.flatten()
print("Shape for X: " + str(X.shape))
print("Shape for y: "+str(y.shape))

X= X[762:,:,:,:]
y= y[762:]
print("New shape for X: " + str(X.shape))
print("New shape for y: "+str(y.shape))
    
dir0= os.path.join("FINAL_regression/right_SMA/output", "forced_filtering_all")
os.mkdir(dir0)
dir1=os.path.join(dir0, "comparison")
dir2=os.path.join(dir0, "temporal_convolution")
dir3=os.path.join(dir0, "spatial_convolution")
os.mkdir(dir1)
os.mkdir(dir2)
os.mkdir(dir3)

################################################################################################################################################################################################################################################################################
Fs=125
EPOCHS=800
lr_test=1e-4
batch_test=64
decay_test=0
kern_test=0.15

n_folds = 5
seed = 21
shuffle_test = True
r2=[]
adj_r2=[]
corr_coef=[]
final_val_loss=[]
stats=[]

kfold = KFold(n_splits = n_folds, shuffle = shuffle_test, random_state = seed)
count=0
sommatoria=0

for train_index, test_index in kfold.split(X):
    count=count+1

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print("Train index for this split: "+ str(train_index)) 
    print("Number of samples for train set: "+str(train_index.shape[0]))
    print("Test index for this split: "+ str(test_index))
    print("Number of samples for test set: "+str(test_index.shape[0]))

    # Define the model architecture - 

    model=Sequential()

    ##################################################################

    model.add(Conv2D(8, (1, 64), padding = 'same',
                                   input_shape = (N_chan, N_samples_long, 1),
                                   use_bias = False, name="temporal_conv"))
    model.add(BatchNormalization(name="batchnorm_1"))
    model.add(DepthwiseConv2D((31, 1), use_bias = False, 
                                   depth_multiplier = 2,
                                   depthwise_constraint = max_norm(1.), name="spatial_conv"))
    model.add(BatchNormalization(name="batchnorm_2"))
    model.add(Activation('elu', name="activation_1"))
    model.add(AveragePooling2D((1, 4), name="pooling_layer_1"))
    model.add(Dropout(0.5, name="dropout_1"))

    model.add(SeparableConv2D(16, (1, 16),
                                   use_bias = False, padding = 'same', name="separable_conv"))
    model.add(BatchNormalization(name="batchnorm_3"))
    model.add(Activation('elu', name="activation_2"))
    model.add(AveragePooling2D((1, 8), name="pooling_layer_2"))
    model.add(Dropout(0.5, name="drpout_2")) #QUI DROPOUT E' LASCIATO A 0.5 come in eegnet paper

    model.add(Flatten(name = 'flatten'))

    model.add(Dense(NUM_ROI, name = 'dense',
                         kernel_constraint = max_norm(kern_test)))
#     model.add(Activation('softmax', name = 'softmax'))


      # Define the optimizer
    optimizer= optimizers.Adam(
    learning_rate= lr_test,
    weight_decay= decay_test
    )
    model.compile(optimizer=optimizer,
                   loss=loss_fn,
                   metrics=[RSquare()])

    evaluation.append(model.fit(X_train, y_train, batch_size=batch_test,
              epochs=EPOCHS, 
              validation_data=(X_test, y_test), 
              verbose=2, workers=1, 
              callbacks=[CustomCallback()]))

    iteration.append(model)
    #SALVO R2 e Adjusted R2 score per fold
    modello= model.predict(X_test)

    temp_r2=[]
    temp_adj_r2=[]
    temp_r2=r2_score(y_test, modello)

    n= y_test.shape[0]
    p= N_chan*N_samples_long
    temp_adj_r2= 1-((1-temp_r2)*((n-1)/(n-p)))

    temp_corr_coef=[]
    modello=modello.flatten()
    temp_corr_coef=np.corrcoef(y_test, modello)[0,1]

    r2.append(temp_r2)
    adj_r2.append(temp_adj_r2)
    corr_coef.append(temp_corr_coef)

    # PLOTTO EFFETTIVA REGRESSIONE VISIVAMENTE
    plt.figure()
    plt.figure(figsize=(25,5))
    plt.plot(y_test)
    plt.plot(model.predict(X_test))
#     plt.plot(y_test-model.predict(X_test))
    plt.savefig(dir1+"/comparison_kfold_"+str(count)+"_zscored_run_clipped")

    var= (model.get_layer("temporal_conv").weights)
    for lallo in range(8):
        plt.figure()
        plt.title("temp_conv_"+str(lallo))
        plt.plot(var[0][0,:,0][:,lallo]) #this way i access the temporal filters, cambiando ultimo zero
        plt.savefig(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
        temp= (var[0][0,:,0][:,lallo]).numpy()
        sio.savemat(dir2+"/ALL_temp_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": temp})
        plt.close()

        neil_x, neil_y=sp.signal.welch(model.trainable_variables[0].numpy()[0,:,0,lallo], Fs, nperseg=64)
        plt.figure()
        plt.title("Spectrogram_"+str(lallo))
        plt.plot(neil_x, neil_y)
        plt.savefig(dir2+"/ALL_spectrogram_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm")
        plt.close()

    var_2= (model.get_layer("spatial_conv").weights)
    reshaped_var_2=tf.reshape(var_2[0][:,0,:,:],(31,16))
    for lallo in range(16):
        nump= reshaped_var_2[:,lallo].numpy()
        sio.savemat(dir3+"/ALL_spat_conv_kfold_"+str(count)+"_filter_"+str(lallo)+"_non_norm.mat", {"array": nump})

# print("--------------------------------------- AVERAGE MEAN AND VARIANCE ACROSS FOLDS ---------------------------------------")
# print(np.mean(val_mean_squared_error))
# print(np.var(val_mean_squared_error))

# PLOTTO LOSS E VALIDATION LOSS, MEAN SQUARED ERROR E VALIDATION MEAN SQUARED ERROR

#plot accuracy and loss function across epochs
epoch_vec=np.linspace(1,EPOCHS,EPOCHS)
min_temp_loss=10
min_temp_acc=10
max_temp_loss=0
max_temp_acc=0

for idx in range(n_folds):
    if (np.min(evaluation[idx].history['loss'])<min_temp_loss):
        min_temp_loss=np.min(evaluation[idx].history['loss'])
    if (np.min(evaluation[idx].history['val_loss'])<min_temp_loss):
        min_temp_loss=np.min(evaluation[idx].history['val_loss'])
    if (np.max(evaluation[idx].history['loss'])>max_temp_loss):
        max_temp_loss=np.max(evaluation[idx].history['loss'])
    if (np.max(evaluation[idx].history['val_loss'])>max_temp_loss):
        max_temp_loss=np.max(evaluation[idx].history['val_loss'])

for idx in range(n_folds):
    if (np.min(evaluation[idx].history['r_square'])<min_temp_acc):
        min_temp_acc=np.min(evaluation[idx].history['r_square'])
    if (np.min(evaluation[idx].history['val_r_square'])<min_temp_acc):
        min_temp_acc=np.min(evaluation[idx].history['val_r_square'])
    if (np.max(evaluation[idx].history['r_square'])>max_temp_acc):
        max_temp_acc=np.max(evaluation[idx].history['r_square'])
    if (np.max(evaluation[idx].history['val_r_square'])>max_temp_acc):
        max_temp_acc=np.max(evaluation[idx].history['val_r_square'])

for idx in range(n_folds):
    loss_vec_train= evaluation[idx].history['loss']
    loss_vec_test= evaluation[idx].history['val_loss']

    plt.figure()
    plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
    plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss across epochs for kfold: '+str(idx))
    plt.ylim([min_temp_loss, max_temp_loss])
    plt.legend()

    plt.savefig(dir1+"/loss_kfold_"+str(idx))
    plt.close()


#plot accuracy and loss function across epochs
epoch_vec=np.linspace(1,EPOCHS,EPOCHS)

inizio=0;
for idx in range(n_folds):
    mse_filter_min= min(MSE_filters_mean)
    mse_filter_max= max(MSE_filters_mean)
    fine=(idx+1)*EPOCHS

    plt.figure()
    plt.plot(MSE_filters_mean[inizio:fine],'b-');

    plt.xlabel('Epoch')
    plt.ylabel('mean mse filters')
    plt.title('mean_mse_temporal_filters across epochs for kfold: '+str(idx))
    plt.ylim([mse_filter_min, mse_filter_max])
#     plt.legend()
    sio.savemat(dir1+"/mean_mse_filter_kfold_"+str(idx)+".mat", {"array": MSE_filters_mean[inizio:fine]})

    plt.savefig(dir1+"/mean_mse_filters_kfold_"+str(idx))
    plt.close()
    inizio=fine

for idx in range(n_folds):
    loss_vec_train= evaluation[idx].history['r_square']
    loss_vec_test= evaluation[idx].history['val_r_square']

    plt.figure()
    plt.plot(epoch_vec,loss_vec_test,'b-', label= 'test');
    plt.plot(epoch_vec,loss_vec_train,'r-', label='train');

    plt.xlabel('Epoch')
    plt.ylabel('r_square')
    plt.title('r_square across epochs for kfold: '+str(idx))
    plt.ylim([min_temp_acc, max_temp_acc])
    plt.legend()

    plt.savefig(dir1+"/r_square_kfold_"+str(idx))
    plt.close()

for idx in range(n_folds):
    temp_val_loss=[]
    temp_val_loss= evaluation[idx].history['val_loss'][-1] #with this indexing i select the last element of the list
    final_val_loss.append(temp_val_loss)

gianfranco= model.predict(X)

plt.figure()
plt.figure(figsize=(25,5))
plt.plot(y, 'b', label='true')
plt.plot(gianfranco, 'r', label='predicted')
plt.legend()
plt.savefig(dir1+"/total_comparison_zscored_run_clipped")
plt.savefig(dir0+"/total_comparison_zscored_run_clipped")

sio.savemat(dir1+"/regression_results_zscored_run_clipped.mat", {"array": gianfranco})

sio.savemat(dir1+"/r2_score_zscored_run_clipped.mat", {"array": r2})
sio.savemat(dir1+"/adjusted_r2_score_zscored_run_clipped.mat", {"array": adj_r2})
sio.savemat(dir1+"/corr_coef_zscored_run_clipped.mat", {"array": corr_coef})
sio.savemat(dir1+"/final_val_loss_zscored_run_clipped.mat", {"array": final_val_loss})

avg_final_loss= np.mean(final_val_loss)
var_final_loss=statistics.pstdev(final_val_loss)
avg_r2= np.mean(r2)
var_r2=statistics.pstdev(r2)
avg_corr_coef= np.mean(corr_coef)
var_corr_coef= statistics.pstdev(corr_coef)

stats.append(avg_final_loss)
stats.append(var_final_loss)
stats.append(avg_r2)
stats.append(var_r2)
stats.append(avg_corr_coef)
stats.append(var_corr_coef)
sio.savemat(dir0+"/stats_zscored_run_clipped.mat", {"array": stats})